In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
time = pd.read_csv('./combined_2016_2018.csv')

In [ ]:
time.head()

Pick one and plot

In [ ]:
locs = list(set(time['localityNo']))

In [ ]:
randomLoc = np.random.choice(locs)
print(randomLoc)
randomLoc = 30977
week = time[time['localityNo'] == randomLoc]['week']
year = time[time['localityNo'] == randomLoc]['year']

hasMechanicalRemoval = np.array(time[time['localityNo'] == randomLoc]['hasMechanicalRemoval'])
hasBathTreatment = np.array(time[time['localityNo'] == randomLoc]['hasBathTreatment'])
hasInFeedTreatment = np.array(time[time['localityNo'] == randomLoc]['hasInFeedTreatment'])
hasCleanerFishDeployed = np.array(time[time['localityNo'] == randomLoc]['hasCleanerFishDeployed'])

weeks = []
for (w, y) in zip(week, year):
    weeks.append(w + (y-2016)*52)

# treatment = hasMechanicalRemoval + hasBathTreatment + hasInFeedTreatment + hasCleanerFishDeployed
# week_treated = np.where(treatment >0)[0]
avgAdultFemaleLice = time[time['localityNo'] == randomLoc]['avgAdultFemaleLice']

In [ ]:
list(avgAdultFemaleLice)

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(weeks, avgAdultFemaleLice, 'k')
for i in range(len(weeks)):
    if hasMechanicalRemoval[i]:
        plt.plot([weeks[i], weeks[i]], [0, 1], 'r')
    if hasBathTreatment[i]:
        plt.plot([weeks[i], weeks[i]], [0, 1], 'g')
    if hasInFeedTreatment[i]:
        plt.plot([weeks[i], weeks[i]], [0, 1], 'b')
    if hasCleanerFishDeployed[i]:
        plt.plot([weeks[i], weeks[i]], [0, 1], 'y')
    # plt.plot([weeks[i], weeks[i]], [0, 1], 'r')
plt.xlim([0, 156])

labels = ['201{}-'.format(j) + str(i) for j in range(6, 9) for i in range(0, 52)]
# plt.xlabel(labels[::4])
plt.xticks(range(0, 156, 4), labels[::4], rotation='vertical')
plt.show()

Plot multiples

In [ ]:
# labels = ['201{}-'.format(j) + str(i) for j in range(6, 9) for i in range(0, 52)]

# for _ in range(10):
#     plt.figure(figsize=(20, 5))
#     randomLoc = np.random.choice(locs)
#     week = time[time['localityNo'] == randomLoc]['week']
#     year = time[time['localityNo'] == randomLoc]['year']
    
#     hasMechanicalRemoval = np.array(time[time['localityNo'] == randomLoc]['hasMechanicalRemoval'])
#     hasBathTreatment = np.array(time[time['localityNo'] == randomLoc]['hasBathTreatment'])
#     hasInFeedTreatment = np.array(time[time['localityNo'] == randomLoc]['hasInFeedTreatment'])
#     hasCleanerFishDeployed = np.array(time[time['localityNo'] == randomLoc]['hasCleanerFishDeployed'])
    
#     weeks = []
#     for (w, y) in zip(week, year):
#         weeks.append(w + (y-2016)*52)
#     avgAdultFemaleLice = time[time['localityNo'] == randomLoc]['avgAdultFemaleLice']
    
#     plt.plot(weeks, avgAdultFemaleLice, 'k')
#     ymax = np.max(avgAdultFemaleLice)
#     for i in range(len(weeks)):
#         if hasMechanicalRemoval[i]:
#             plt.plot([weeks[i], weeks[i]], [0, ymax], 'r')
#         if hasBathTreatment[i]:
#             plt.plot([weeks[i], weeks[i]], [0, ymax], 'g')
#         if hasInFeedTreatment[i]:
#             plt.plot([weeks[i], weeks[i]], [0, ymax], 'b')
#         if hasCleanerFishDeployed[i]:
#             plt.plot([weeks[i], weeks[i]], [0, ymax], 'y')
#     plt.xlim([0, 156])
#     plt.xticks(range(0, 156, 4), labels[::4], rotation='vertical')
#     plt.show()

Create dataset

In [ ]:
import random
from sklearn.preprocessing import MinMaxScaler
random.seed(0)

some stuff

In [ ]:
n_in = 10 # input sequence length
n_pred = 4 # predictions for 4 weeks
locs = list(set(time['localityNo'])) 
print("Number of unique location : {}".format(len(locs)))
random.shuffle(locs)
cutoff = int(0.8*len(locs))
train = locs[:cutoff]
test = locs[cutoff:]
print('Number of train locs: {}'.format(len(train)))
print('Number of test locs: {}'.format(len(test)))

function to create dataset

In [ ]:
def find_sequence(data, length):
    all_seqs = []
    seq = [0]
    for i in range(1, len(data)):
        if len(seq) == length:
            all_seqs.append(seq)
            seq = seq[1:]
        if data[i]-1 == data[i-1]:
            # seq 
            seq.append(i)
        else:
            seq = [i]
#     if len(seq) == length:
#         all_seqs.append(seq)
    return all_seqs

In [ ]:
find_sequence(list(range(10, 20))+list(range(30, 40)), 8)

In [ ]:
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return np.array(diff)

In [ ]:
def create_supervised(df, loc, n_in=8, n_pred=4):
    week = df[df['localityNo'] == loc]['week']
    year = df[df['localityNo'] == loc]['year']
    avgAdultFemaleLice = list(df[df['localityNo'] == loc]['avgAdultFemaleLice'])

    # remove trend
    avgAdultFemaleLice = difference(avgAdultFemaleLice)
    if len(avgAdultFemaleLice) == 0:
        return [], []
    # scale
    scaler = MinMaxScaler(feature_range=(0, 1))
    avgAdultFemaleLice = scaler.fit_transform(np.expand_dims(avgAdultFemaleLice, axis=1))
    
    # get the weeks
    weeks = []
    for (w, ye) in zip(week, year):
        weeks.append(w + (ye-2016)*52)
    
    # find all the sequences of length n_in + n_pred
    consec_weeks = find_sequence(weeks, n_in+n_pred)
    # print(len(consec_weeks))
    X = []
    y = []
    for we in consec_weeks:
        X.append([avgAdultFemaleLice[idx]  for idx in we[:n_in]])
        y.append([avgAdultFemaleLice[idx]  for idx in we[n_in:]])
    
    return np.array(X), np.array(y)

In [ ]:
consec_weeks = find_sequence(weeks, 8)

In [ ]:
# test on one example
x, y = create_supervised(time, 10054, 10)
print(x.shape)
plt.plot(range(10), x[0, :])
plt.plot(range(10, 14, 1), y[0, :], 'r')
plt.show()

Create model